In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from threading import Lock
import time
import re
from pprint import pprint

# options = Options()
# options.add_argument('--headless')
# browser = webdriver.Chrome(options=options)
browser = webdriver.Chrome()

deepl_url = 'https://www.deepl.com/ja/translator'

eg = "apple and orange, i like them. but i don't like banana 😆"
# eg = '⚪📢😆'

browser.get(deepl_url)

css_selector_for_stextarea = """
#headlessui-tabs-panel-7 > div > div.border-dark-7.border.bg-white.shadow-sm.xl\:rounded-lg > section > div > div.rounded-bl-inherit.rounded-br-inherit.grid.grid-cols-1.grid-rows-\[auto_auto\].min-\[768px\]\:grid-cols-\[1fr_auto_1fr\].min-\[768px\]\:grid-rows-\[1fr\].TextTranslatorLayout-module--bothareas--wXSVy > div.rounded-bl-inherit.relative.z-\[1\].min-h-\[240px\].min-w-0.md\:min-h-\[clamp\(250px\,50vh\,557px\)\] > section > div > div.relative.flex-1 > d-textarea > div:nth-child(1)
"""

css_selector_for_ttextarea = """
#headlessui-tabs-panel-7 > div > div.border-dark-7.border.bg-white.shadow-sm.xl\:rounded-lg > section > div > div.rounded-bl-inherit.rounded-br-inherit.grid.grid-cols-1.grid-rows-\[auto_auto\].min-\[768px\]\:grid-cols-\[1fr_auto_1fr\].min-\[768px\]\:grid-rows-\[1fr\].TextTranslatorLayout-module--bothareas--wXSVy > div.rounded-br-inherit.relative.z-\[1\].min-h-\[240px\].min-w-0.md\:min-h-\[clamp\(250px\,50vh\,557px\)\].max-\[768px\]\:min-h-\[375px\] > section > div.relative.flex.flex-1.flex-col > d-textarea > div
"""
lock = Lock()
lock.acquire()

stextarea = browser.find_element(By.CSS_SELECTOR, css_selector_for_stextarea)
ttextarea = browser.find_element(By.CSS_SELECTOR, css_selector_for_ttextarea)

# stextarea.send_keys(eg)

# INPUT_EMOJI = """
# arguments[0].value += arguments[1];
# arguments[0].dispatchEvent(new Event('change'));
# """

INPUT_EMOJI = 'arguments[0].innerHTML = "{}"'.format(eg)

# INPUT_EMOJI = '''
# arguments[0].innerHTML = arguments[1];
# arguments[0].dispatchEvent(new Event('change'));
# '''

# browser.execute_script(INPUT_EMOJI, stextarea, eg)
# stextarea.send_keys(Keys.ENTER)

browser.execute_script(INPUT_EMOJI, stextarea)
stextarea.send_keys('.')
stextarea.send_keys(Keys.BACKSPACE)

lock.release()
            #time.sleep(2)

translated_text = ''
translated_texts = []

lock.acquire()
# 完全に翻訳されるまで繰り返す。判定は翻訳後文章の文字数が0かどうか
# 文字数が0の場合は繰り返される。
sec = 2

stop_iter = 4
iter = 0
while not translated_text:
    # print("keep now....")
    time.sleep(sec)
    translated_text = ttextarea.text
    print(f"translated_text: {translated_text}")
    # print("translated_text: \n{}\n len: {}\n ".format(translated_text, len(translated_text))) 
    can_translated = re.search("\[\.\.\.\]", translated_text)
    if type(can_translated) is re.Match:
        print("\ntry again\n")
        translated_text = ''
        sec += 2
    elif len(translated_text) == 0: # 翻訳されていない場合（[...]すら表示されていないパターン）
        print("\ntry again\n")
        translated_text = ''
        sec += 2
    else:
        translated_texts.append(translated_text)

    iter += 1
    if iter > stop_iter:
        break
# ここに翻訳結果が出力されたら成功（途中経過をjupyter notebookで確認する場合はコメントイン）
#print(translated_text)    
lock.release()

# sourse_textareaをクリアする
time.sleep(2)
stextarea.send_keys(Keys.CONTROL, "a")
stextarea.send_keys(Keys.BACKSPACE)


# 翻訳結果を出力
for translated_text in translated_texts:
    pprint(translated_text)

translated_text: 

try again

translated_text: リンゴとオレンジは好きだけど、バナナは苦手😆。
'リンゴとオレンジは好きだけど、バナナは苦手😆。'


In [5]:
type(stextarea)

selenium.webdriver.remote.webelement.WebElement

In [ ]:
browser.execute_script

In [4]:
dir(ttextarea)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_execute',
 '_id',
 '_parent',
 '_upload',
 'accessible_name',
 'aria_role',
 'clear',
 'click',
 'find_element',
 'find_elements',
 'get_attribute',
 'get_dom_attribute',
 'get_property',
 'id',
 'is_displayed',
 'is_enabled',
 'is_selected',
 'location',
 'location_once_scrolled_into_view',
 'parent',
 'rect',
 'screenshot',
 'screenshot_as_base64',
 'screenshot_as_png',
 'send_keys',
 'shadow_root',
 'size',
 'submit',
 'tag_name',
 'text',
 'value_of_css_property']

In [5]:
ttextarea.text

'アップル'

In [ ]:
"""
//*[@id="headlessui-tabs-panel-7"]/div/div[1]/section/div/div[2]/div[1]/section/div/div[1]/d-textarea/div[1]/p
"""

"""
#headlessui-tabs-panel-7 > div > div.border-dark-7.border.bg-white.shadow-sm.xl\:rounded-lg > section > div > div.rounded-bl-inherit.rounded-br-inherit.grid.grid-cols-1.grid-rows-\[auto_auto\].min-\[768px\]\:grid-cols-\[1fr_auto_1fr\].min-\[768px\]\:grid-rows-\[1fr\].TextTranslatorLayout-module--bothareas--wXSVy > div.rounded-bl-inherit.relative.z-\[1\].min-h-\[240px\].min-w-0.md\:min-h-\[clamp\(250px\,50vh\,557px\)\] > section > div > div.relative.flex-1 > d-textarea > div:nth-child(1)
"""